In [1]:
import numpy as np
import pandas as pd
import novosparc
from spaotsc import SpaOTsc
import tangram as tg

import anndata as ad
import scanpy as sc

from sklearn.model_selection import KFold

import os
import time

from SC2Spa import SI, PP, Vis

/mnt/win1
/mnt/win1/SC2Spa_Notebook/DroEmbryo


In [2]:
SC = pd.read_csv('dge_raw.txt', sep = '\t', header = None, index_col = 0)
ST = pd.read_csv('bdtnp.txt', sep = '\t')
loc = pd.read_csv('geometry.txt', sep = ' ')

In [3]:
print(SC.shape)
SC.head()

(8924, 1297)


,1,2,3,4,5,6,7,8,9,10,...,1288,1289,1290,1291,1292,1293,1294,1295,1296,1297
0,,,,,,,,,,,,,,,,,,,,,
128up,4,2,0,0,0,0,0,1,1,1,...,0,0,1,0,0,0,0,1,0,0
140up,1,3,0,0,0,2,2,3,0,0,...,1,0,1,2,1,1,1,2,0,0
14-3-3epsilon,370,188,115,119,92,120,107,95,84,104,...,33,27,52,97,73,55,27,110,47,45
14-3-3zeta,49,18,16,8,18,13,12,8,21,16,...,2,4,12,15,13,11,5,12,3,8
18w,8,3,1,4,3,1,2,1,0,0,...,1,0,0,0,2,0,0,0,6,2


In [4]:
print(ST.shape)
ST.head()

(3039, 84)


,aay,Ama,Ance,Antp,apt,Blimp-1,bmm,bowl,brk,Btk29A,...,tkv,tll,toc,Traf4,trn,tsh,twi,zen,zen2,zfh1
0,0.116516,0.224190,0.033730,0.104307,0.673878,0.856570,0.469676,0.286151,0.320923,0.737946,...,0.353227,0.102896,0.153453,0.129360,0.087361,0.147089,0.225082,0.027564,0.297847,0.372422
1,0.113320,0.208761,0.044785,0.103447,0.743820,0.462554,0.570401,0.345240,0.320727,0.913163,...,0.268072,0.093506,0.206405,0.313327,0.230506,0.190749,0.440296,0.024157,0.455723,0.389489
2,0.092872,0.172598,0.036166,0.096985,0.838291,0.467538,0.478229,0.377768,0.297598,0.931254,...,0.315466,0.090488,0.196441,0.161869,0.102645,0.177968,0.347098,0.019817,0.598716,0.502384
3,0.101251,0.207963,0.066009,0.092872,0.597823,0.497481,0.479122,0.580058,0.334111,0.746340,...,0.211007,0.110080,0.205833,0.287976,0.382287,0.189138,0.542763,0.019013,0.570066,0.397117
4,0.127009,0.176440,0.084271,0.027938,0.621983,0.833522,0.680243,0.201147,0.359235,0.755608,...,0.447295,0.079012,0.152232,0.105062,0.109857,0.166762,0.259750,0.025874,0.349462,0.498263


In [5]:
print(loc.shape)
loc.head()

(3039, 3)


,x,y,z
0,-194.042242,0.221890,30.195476
1,-203.322900,1.928557,20.081308
2,-200.250107,3.588368,23.932470
3,-205.875337,3.577224,14.820966
4,-193.173936,9.531855,29.667388


In [6]:
loc_Norm = PP.MinMaxNorm(loc, Y_ref = loc)
print(loc_Norm.shape)
loc_Norm.head()

(3039, 3)


,x,y,z
0,0.034029,0.460993,0.748371
1,0.010344,0.473811,0.678778
2,0.018186,0.486276,0.705277
3,0.003830,0.486193,0.642582
4,0.036245,0.530913,0.744737


In [7]:
adata = ad.AnnData(X=ST.values, var = pd.DataFrame(ST.columns).set_index(0))
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

X = adata.X
Y = loc_Norm.values

# Split groups

In [8]:
'''
root = 'CV_groups/'
if not os.path.exists(root):
    os.makedirs(root)
    
kf = KFold(n_splits=5, random_state = 2022, shuffle = True)
cv_index = kf.split(X)

i = 0
for cv_train, cv_test in cv_index:
    i+=1
    pd.DataFrame(cv_train).to_csv(root + 'index_train_' + str(i) + '.csv', header = None)
    pd.DataFrame(cv_test).to_csv(root + 'index_test_' + str(i) + '.csv', header = None)

'''

train_indices, test_indices = PP.LoadIndices()

# SC2Spa

In [9]:
sta = time.time()

history, train_preds, test_preds = SI.CrossValidation(X=X, Y=Y,
                                          train_indices = train_indices, test_indices = test_indices,
                                          epoch = 500,
                                          batch_size = 512,
                                          nodes = [64, 32, 16, 8, 4], seed = None)

end = time.time()
print((end - sta) / 60.0, 'min')

0


2022-06-21 16:02:41.678120: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-21 16:02:42.395407: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22031 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:3b:00.0, compute capability: 7.5
/mnt/win1/SC2Spa/SI.py:112: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator = batch_generator(X, Y, cv_train,


Epoch 1/500
4/4 [==============================] - 3s 127ms/step - loss: 0.3218 - rmse: 0.3130 - val_loss: 0.3574 - val_rmse: 0.3487 - lr: 0.0010
Epoch 2/500
4/4 [==============================] - 0s 49ms/step - loss: 0.3081 - rmse: 0.2994 - val_loss: 0.3556 - val_rmse: 0.3469 - lr: 0.0010
Epoch 3/500
4/4 [==============================] - 0s 49ms/step - loss: 0.2963 - rmse: 0.2877 - val_loss: 0.3575 - val_rmse: 0.3488 - lr: 0.0010
Epoch 4/500
4/4 [==============================] - 0s 48ms/step - loss: 0.2899 - rmse: 0.2812 - val_loss: 0.3627 - val_rmse: 0.3541 - lr: 0.0010
Epoch 5/500
4/4 [==============================] - 0s 48ms/step - loss: 0.2802 - rmse: 0.2716 - val_loss: 0.3745 - val_rmse: 0.3659 - lr: 0.0010
Epoch 6/500
4/4 [==============================] - 0s 47ms/step - loss: 0.2737 - rmse: 0.2651 - val_loss: 0.3818 - val_rmse: 0.3732 - lr: 0.0010
Epoch 7/500
4/4 [==============================] - 0s 49ms/step - loss: 0.2655 - rmse: 0.2569 - val_loss: 0.3856 - val_rmse: 0.37

4/4 [==============================] - 0s 39ms/step - loss: 0.0920 - rmse: 0.0843 - val_loss: 0.0542 - val_rmse: 0.0466 - lr: 0.0010
Epoch 114/500
4/4 [==============================] - 0s 38ms/step - loss: 0.0934 - rmse: 0.0858 - val_loss: 0.0539 - val_rmse: 0.0462 - lr: 0.0010
Epoch 115/500
4/4 [==============================] - 0s 41ms/step - loss: 0.0928 - rmse: 0.0851 - val_loss: 0.0549 - val_rmse: 0.0472 - lr: 0.0010
Epoch 116/500
4/4 [==============================] - 0s 36ms/step - loss: 0.0958 - rmse: 0.0882 - val_loss: 0.0549 - val_rmse: 0.0473 - lr: 0.0010
Epoch 117/500
4/4 [==============================] - 0s 39ms/step - loss: 0.0932 - rmse: 0.0855 - val_loss: 0.0567 - val_rmse: 0.0491 - lr: 0.0010
Epoch 118/500
4/4 [==============================] - 0s 36ms/step - loss: 0.0920 - rmse: 0.0844 - val_loss: 0.0602 - val_rmse: 0.0526 - lr: 0.0010
Epoch 119/500
4/4 [==============================] - 0s 48ms/step - loss: 0.0907 - rmse: 0.0831 - val_loss: 0.0685 - val_rmse: 0.060

4/4 [==============================] - 0s 39ms/step - loss: 0.0790 - rmse: 0.0718 - val_loss: 0.0447 - val_rmse: 0.0375 - lr: 1.2500e-04
Epoch 219: early stopping
1
Epoch 1/500
4/4 [==============================] - 2s 96ms/step - loss: 0.3359 - rmse: 0.3270 - val_loss: 0.2933 - val_rmse: 0.2844 - lr: 0.0010
Epoch 2/500
4/4 [==============================] - 0s 33ms/step - loss: 0.3173 - rmse: 0.3084 - val_loss: 0.2878 - val_rmse: 0.2789 - lr: 0.0010
Epoch 3/500
4/4 [==============================] - 0s 46ms/step - loss: 0.2997 - rmse: 0.2908 - val_loss: 0.3064 - val_rmse: 0.2975 - lr: 0.0010
Epoch 4/500
4/4 [==============================] - 0s 45ms/step - loss: 0.2896 - rmse: 0.2807 - val_loss: 0.2932 - val_rmse: 0.2843 - lr: 0.0010
Epoch 5/500
4/4 [==============================] - 0s 46ms/step - loss: 0.2806 - rmse: 0.2718 - val_loss: 0.2845 - val_rmse: 0.2757 - lr: 0.0010
Epoch 6/500
4/4 [==============================] - 0s 43ms/step - loss: 0.2714 - rmse: 0.2625 - val_loss: 0.27

...

Epoch 139/500
4/4 [==============================] - 0s 45ms/step - loss: 0.0919 - rmse: 0.0847 - val_loss: 0.0608 - val_rmse: 0.0535 - lr: 0.0010
Epoch 140/500
4/4 [==============================] - 0s 39ms/step - loss: 0.0881 - rmse: 0.0809 - val_loss: 0.0610 - val_rmse: 0.0537 - lr: 0.0010
Epoch 141/500
4/4 [==============================] - 0s 35ms/step - loss: 0.0902 - rmse: 0.0830 - val_loss: 0.0481 - val_rmse: 0.0408 - lr: 0.0010
Epoch 142/500
4/4 [==============================] - 0s 46ms/step - loss: 0.0910 - rmse: 0.0838 - val_loss: 0.0570 - val_rmse: 0.0498 - lr: 0.0010
Epoch 143/500
4/4 [==============================] - 0s 46ms/step - loss: 0.0889 - rmse: 0.0817 - val_loss: 0.0670 - val_rmse: 0.0598 - lr: 0.0010
Epoch 144/500
4/4 [==============================] - 0s 46ms/step - loss: 0.0871 - rmse: 0.0799 - val_loss: 0.0673 - val_rmse: 0.0601 - lr: 0.0010
Epoch 145/500
4/4 [==============================] - 0s 42ms/step - loss: 0.0886 - rmse: 0.0814 - val_loss: 0.0641 - v

4/4 [==============================] - 0s 47ms/step - loss: 0.0788 - rmse: 0.0720 - val_loss: 0.0436 - val_rmse: 0.0368 - lr: 2.5000e-04
Epoch 247/500
4/4 [==============================] - 0s 36ms/step - loss: 0.0784 - rmse: 0.0716 - val_loss: 0.0427 - val_rmse: 0.0359 - lr: 2.5000e-04
Epoch 248/500
4/4 [==============================] - 0s 49ms/step - loss: 0.0772 - rmse: 0.0704 - val_loss: 0.0423 - val_rmse: 0.0355 - lr: 2.5000e-04
Epoch 249/500
4/4 [==============================] - 0s 47ms/step - loss: 0.0783 - rmse: 0.0715 - val_loss: 0.0421 - val_rmse: 0.0353 - lr: 2.5000e-04
Epoch 250/500
4/4 [==============================] - 0s 45ms/step - loss: 0.0784 - rmse: 0.0716 - val_loss: 0.0425 - val_rmse: 0.0357 - lr: 2.5000e-04
Epoch 251/500
4/4 [==============================] - 0s 46ms/step - loss: 0.0779 - rmse: 0.0711 - val_loss: 0.0426 - val_rmse: 0.0358 - lr: 2.5000e-04
Epoch 252/500
4/4 [==============================] - 0s 40ms/step - loss: 0.0804 - rmse: 0.0736 - val_loss: 

4/4 [==============================] - 0s 48ms/step - loss: 0.3225 - rmse: 0.3138 - val_loss: 0.2971 - val_rmse: 0.2884 - lr: 0.0010
Epoch 4/500
4/4 [==============================] - 0s 49ms/step - loss: 0.3084 - rmse: 0.2997 - val_loss: 0.2799 - val_rmse: 0.2712 - lr: 0.0010
Epoch 5/500
4/4 [==============================] - 0s 49ms/step - loss: 0.2967 - rmse: 0.2880 - val_loss: 0.2666 - val_rmse: 0.2580 - lr: 0.0010
Epoch 6/500
4/4 [==============================] - 0s 49ms/step - loss: 0.2922 - rmse: 0.2836 - val_loss: 0.2592 - val_rmse: 0.2506 - lr: 0.0010
Epoch 7/500
4/4 [==============================] - 0s 49ms/step - loss: 0.2846 - rmse: 0.2760 - val_loss: 0.2506 - val_rmse: 0.2420 - lr: 0.0010
Epoch 8/500
4/4 [==============================] - 0s 47ms/step - loss: 0.2752 - rmse: 0.2667 - val_loss: 0.2371 - val_rmse: 0.2285 - lr: 0.0010
Epoch 9/500
4/4 [==============================] - 0s 48ms/step - loss: 0.2696 - rmse: 0.2611 - val_loss: 0.2252 - val_rmse: 0.2167 - lr: 0.00

4/4 [==============================] - 0s 47ms/step - loss: 0.0969 - rmse: 0.0894 - val_loss: 0.0528 - val_rmse: 0.0453 - lr: 0.0010
Epoch 116/500
4/4 [==============================] - 0s 49ms/step - loss: 0.0956 - rmse: 0.0881 - val_loss: 0.0558 - val_rmse: 0.0483 - lr: 0.0010
Epoch 117/500
4/4 [==============================] - 0s 49ms/step - loss: 0.0960 - rmse: 0.0885 - val_loss: 0.0652 - val_rmse: 0.0577 - lr: 0.0010
Epoch 118/500
4/4 [==============================] - 0s 50ms/step - loss: 0.0946 - rmse: 0.0871 - val_loss: 0.0544 - val_rmse: 0.0469 - lr: 0.0010
Epoch 119/500
4/4 [==============================] - 0s 51ms/step - loss: 0.0945 - rmse: 0.0870 - val_loss: 0.0627 - val_rmse: 0.0552 - lr: 0.0010
Epoch 120/500
4/4 [==============================] - 0s 48ms/step - loss: 0.0957 - rmse: 0.0883 - val_loss: 0.0718 - val_rmse: 0.0644 - lr: 0.0010
Epoch 121/500
4/4 [==============================] - 0s 46ms/step - loss: 0.0973 - rmse: 0.0899 - val_loss: 0.0769 - val_rmse: 0.069

Epoch 171/500
4/4 [==============================] - 0s 47ms/step - loss: 0.0871 - rmse: 0.0799 - val_loss: 0.0777 - val_rmse: 0.0705 - lr: 0.0010
Epoch 172/500
4/4 [==============================] - 0s 49ms/step - loss: 0.0861 - rmse: 0.0790 - val_loss: 0.0660 - val_rmse: 0.0589 - lr: 0.0010
Epoch 173/500
4/4 [==============================] - 0s 51ms/step - loss: 0.0859 - rmse: 0.0788 - val_loss: 0.0567 - val_rmse: 0.0496 - lr: 0.0010
Epoch 174/500
5/4 [===============================] - ETA: 0s - loss: 0.0872 - rmse: 0.0801
Epoch 174: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
4/4 [==============================] - 0s 51ms/step - loss: 0.0872 - rmse: 0.0801 - val_loss: 0.0559 - val_rmse: 0.0487 - lr: 0.0010
Epoch 175/500
4/4 [==============================] - 0s 44ms/step - loss: 0.0868 - rmse: 0.0797 - val_loss: 0.0547 - val_rmse: 0.0476 - lr: 5.0000e-04
Epoch 176/500
4/4 [==============================] - 0s 46ms/step - loss: 0.0870 - rmse: 0.0798 - val_los

In [12]:
root = 'SI_Benchmarking/SC2Spa/'
if not os.path.exists(root):
    os.makedirs(root)
    
for i in range(5):
    pd.DataFrame(PP.ReMMNorm(loc.values, test_preds[i])).to_csv('SI_Benchmarking/SC2Spa/pred_' + str(i) + '.csv', header = None)

# LR

In [13]:
sta = time.time()

history, train_preds, test_preds = SI.CV_LR(X=X, Y=Y,
                                         train_indices = train_indices, 
                                         test_indices = test_indices, l1_reg=1e-5, l2_reg = 0,
                                         dropout = 0.05, epoch = 500, batch_size = 512)

end = time.time()
print((end - sta) / 60.0, 'min')

0
Epoch 1/500


/mnt/win1/SC2Spa/SI.py:552: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator = batch_generator(X,\


4/4 [==============================] - 1s 55ms/step - loss: 0.6123 - rmse: 0.6120 - val_loss: 0.5872 - val_rmse: 0.5869 - lr: 0.0010
Epoch 2/500
4/4 [==============================] - 0s 31ms/step - loss: 0.5862 - rmse: 0.5859 - val_loss: 0.5586 - val_rmse: 0.5583 - lr: 0.0010
Epoch 3/500
4/4 [==============================] - 0s 30ms/step - loss: 0.5509 - rmse: 0.5506 - val_loss: 0.4894 - val_rmse: 0.4891 - lr: 0.0010
Epoch 4/500
4/4 [==============================] - 0s 27ms/step - loss: 0.4739 - rmse: 0.4736 - val_loss: 0.3608 - val_rmse: 0.3605 - lr: 0.0010
Epoch 5/500
4/4 [==============================] - 0s 28ms/step - loss: 0.3949 - rmse: 0.3946 - val_loss: 0.3315 - val_rmse: 0.3312 - lr: 0.0010
Epoch 6/500
4/4 [==============================] - 0s 29ms/step - loss: 0.3460 - rmse: 0.3457 - val_loss: 0.2624 - val_rmse: 0.2621 - lr: 0.0010
Epoch 7/500
4/4 [==============================] - 0s 29ms/step - loss: 0.3154 - rmse: 0.3151 - val_loss: 0.2900 - val_rmse: 0.2897 - lr: 0.00

Epoch 55/500
4/4 [==============================] - 0s 32ms/step - loss: 0.2535 - rmse: 0.2532 - val_loss: 0.1723 - val_rmse: 0.1720 - lr: 5.0000e-04
Epoch 56/500
4/4 [==============================] - 0s 29ms/step - loss: 0.2558 - rmse: 0.2555 - val_loss: 0.1701 - val_rmse: 0.1698 - lr: 5.0000e-04
Epoch 57/500
4/4 [==============================] - 0s 29ms/step - loss: 0.2548 - rmse: 0.2545 - val_loss: 0.1710 - val_rmse: 0.1707 - lr: 5.0000e-04
Epoch 58/500
4/4 [==============================] - 0s 26ms/step - loss: 0.2517 - rmse: 0.2514 - val_loss: 0.1700 - val_rmse: 0.1697 - lr: 5.0000e-04
Epoch 59/500
4/4 [==============================] - 0s 25ms/step - loss: 0.2504 - rmse: 0.2501 - val_loss: 0.1661 - val_rmse: 0.1658 - lr: 5.0000e-04
Epoch 60/500
4/4 [==============================] - 0s 32ms/step - loss: 0.2478 - rmse: 0.2475 - val_loss: 0.1649 - val_rmse: 0.1646 - lr: 5.0000e-04
Epoch 61/500
4/4 [==============================] - 0s 29ms/step - loss: 0.2436 - rmse: 0.2433 - val

4/4 [==============================] - 0s 28ms/step - loss: 0.2228 - rmse: 0.2225 - val_loss: 0.1355 - val_rmse: 0.1352 - lr: 3.0518e-08
Epoch 149/500
4/4 [==============================] - 0s 23ms/step - loss: 0.2229 - rmse: 0.2226 - val_loss: 0.1355 - val_rmse: 0.1352 - lr: 1.5259e-08
Epoch 150/500
4/4 [==============================] - 0s 28ms/step - loss: 0.2207 - rmse: 0.2205 - val_loss: 0.1355 - val_rmse: 0.1352 - lr: 1.5259e-08
Epoch 151/500
1/4 [=====>........................] - ETA: 0s - loss: 0.2207 - rmse: 0.2204
Epoch 151: ReduceLROnPlateau reducing learning rate to 1e-08.
4/4 [==============================] - 0s 33ms/step - loss: 0.2228 - rmse: 0.2225 - val_loss: 0.1355 - val_rmse: 0.1352 - lr: 1.5259e-08
Epoch 152/500
4/4 [==============================] - 0s 27ms/step - loss: 0.2264 - rmse: 0.2261 - val_loss: 0.1355 - val_rmse: 0.1352 - lr: 1.0000e-08
Epoch 153/500
4/4 [==============================] - 0s 26ms/step - loss: 0.2224 - rmse: 0.2221 - val_loss: 0.1355 - val

...

4/4 [==============================] - 0s 29ms/step - loss: 0.2080 - rmse: 0.2077 - val_loss: 0.1339 - val_rmse: 0.1337 - lr: 1.0000e-08
Epoch 135/500
4/4 [==============================] - 0s 26ms/step - loss: 0.2091 - rmse: 0.2088 - val_loss: 0.1339 - val_rmse: 0.1337 - lr: 1.0000e-08
Epoch 136/500
4/4 [==============================] - 0s 28ms/step - loss: 0.2056 - rmse: 0.2053 - val_loss: 0.1339 - val_rmse: 0.1337 - lr: 1.0000e-08
Epoch 137/500
4/4 [==============================] - 0s 27ms/step - loss: 0.2076 - rmse: 0.2073 - val_loss: 0.1339 - val_rmse: 0.1337 - lr: 1.0000e-08
Epoch 138/500
4/4 [==============================] - 0s 27ms/step - loss: 0.2096 - rmse: 0.2094 - val_loss: 0.1339 - val_rmse: 0.1337 - lr: 1.0000e-08
Epoch 138: early stopping
3
Epoch 1/500
4/4 [==============================] - 1s 56ms/step - loss: 0.3715 - rmse: 0.3712 - val_loss: 0.3132 - val_rmse: 0.3129 - lr: 0.0010
Epoch 2/500
4/4 [==============================] - 0s 25ms/step - loss: 0.3277 - rmse:

Epoch 52/500
4/4 [==============================] - 0s 22ms/step - loss: 0.2003 - rmse: 0.2000 - val_loss: 0.1149 - val_rmse: 0.1147 - lr: 0.0010
Epoch 53/500
4/4 [==============================] - 0s 22ms/step - loss: 0.1941 - rmse: 0.1939 - val_loss: 0.1096 - val_rmse: 0.1094 - lr: 0.0010
Epoch 54/500
4/4 [==============================] - 0s 32ms/step - loss: 0.1988 - rmse: 0.1986 - val_loss: 0.1093 - val_rmse: 0.1090 - lr: 0.0010
Epoch 55/500
4/4 [==============================] - 0s 29ms/step - loss: 0.1969 - rmse: 0.1966 - val_loss: 0.1066 - val_rmse: 0.1063 - lr: 0.0010
Epoch 56/500
4/4 [==============================] - 0s 30ms/step - loss: 0.1967 - rmse: 0.1965 - val_loss: 0.1079 - val_rmse: 0.1077 - lr: 0.0010
Epoch 57/500
4/4 [==============================] - 0s 31ms/step - loss: 0.1962 - rmse: 0.1960 - val_loss: 0.1031 - val_rmse: 0.1029 - lr: 0.0010
Epoch 58/500
4/4 [==============================] - 0s 30ms/step - loss: 0.1902 - rmse: 0.1900 - val_loss: 0.1032 - val_rmse

4/4 [==============================] - 0s 31ms/step - loss: 0.1692 - rmse: 0.1690 - val_loss: 0.0857 - val_rmse: 0.0854 - lr: 3.9063e-06
Epoch 99/500
4/4 [==============================] - 0s 30ms/step - loss: 0.1695 - rmse: 0.1693 - val_loss: 0.0857 - val_rmse: 0.0854 - lr: 1.9531e-06
Epoch 100/500
4/4 [==============================] - 0s 31ms/step - loss: 0.1684 - rmse: 0.1682 - val_loss: 0.0857 - val_rmse: 0.0855 - lr: 1.9531e-06
Epoch 101/500
1/4 [=====>........................] - ETA: 0s - loss: 0.1673 - rmse: 0.1671
Epoch 101: ReduceLROnPlateau reducing learning rate to 9.765625463842298e-07.
4/4 [==============================] - 0s 31ms/step - loss: 0.1685 - rmse: 0.1683 - val_loss: 0.0857 - val_rmse: 0.0855 - lr: 1.9531e-06
Epoch 102/500
4/4 [==============================] - 0s 27ms/step - loss: 0.1681 - rmse: 0.1679 - val_loss: 0.0857 - val_rmse: 0.0855 - lr: 9.7656e-07
Epoch 103/500
4/4 [==============================] - 0s 26ms/step - loss: 0.1706 - rmse: 0.1704 - val_los

Epoch 21/500
4/4 [==============================] - 0s 32ms/step - loss: 0.2863 - rmse: 0.2860 - val_loss: 0.2202 - val_rmse: 0.2199 - lr: 0.0010
Epoch 22/500
4/4 [==============================] - 0s 31ms/step - loss: 0.2821 - rmse: 0.2818 - val_loss: 0.2172 - val_rmse: 0.2169 - lr: 0.0010
Epoch 23/500
4/4 [==============================] - 0s 32ms/step - loss: 0.2807 - rmse: 0.2804 - val_loss: 0.2081 - val_rmse: 0.2078 - lr: 0.0010
Epoch 24/500
4/4 [==============================] - 0s 31ms/step - loss: 0.2807 - rmse: 0.2804 - val_loss: 0.2109 - val_rmse: 0.2106 - lr: 0.0010
Epoch 25/500
4/4 [==============================] - 0s 32ms/step - loss: 0.2748 - rmse: 0.2745 - val_loss: 0.2041 - val_rmse: 0.2038 - lr: 0.0010
Epoch 26/500
4/4 [==============================] - 0s 32ms/step - loss: 0.2718 - rmse: 0.2715 - val_loss: 0.1970 - val_rmse: 0.1967 - lr: 0.0010
Epoch 27/500
4/4 [==============================] - 0s 31ms/step - loss: 0.2692 - rmse: 0.2689 - val_loss: 0.1985 - val_rmse

Epoch 75/500
4/4 [==============================] - 0s 28ms/step - loss: 0.1815 - rmse: 0.1812 - val_loss: 0.0947 - val_rmse: 0.0944 - lr: 2.5000e-04
Epoch 76/500
4/4 [==============================] - 0s 29ms/step - loss: 0.1845 - rmse: 0.1842 - val_loss: 0.0971 - val_rmse: 0.0969 - lr: 2.5000e-04
Epoch 77/500
4/4 [==============================] - 0s 29ms/step - loss: 0.1848 - rmse: 0.1845 - val_loss: 0.0963 - val_rmse: 0.0961 - lr: 2.5000e-04
Epoch 78/500
1/4 [=====>........................] - ETA: 0s - loss: 0.1851 - rmse: 0.1849
Epoch 78: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
4/4 [==============================] - 0s 30ms/step - loss: 0.1842 - rmse: 0.1839 - val_loss: 0.0951 - val_rmse: 0.0948 - lr: 2.5000e-04
Epoch 79/500
4/4 [==============================] - 0s 30ms/step - loss: 0.1833 - rmse: 0.1831 - val_loss: 0.0949 - val_rmse: 0.0947 - lr: 1.2500e-04
Epoch 80/500
4/4 [==============================] - 0s 26ms/step - loss: 0.1825 - rmse: 0.1823 -

In [14]:
root = 'SI_Benchmarking/LR/'
if not os.path.exists(root):
    os.makedirs(root)

for i in range(5):
    pd.DataFrame(PP.ReMMNorm(loc.values, test_preds[i])).to_csv('SI_Benchmarking/LR/pred_' + str(i) + '.csv', header = None)

# NovoSpaRc

In [54]:
root = 'SI_Benchmarking/novosparc/'
if not os.path.exists(root):
    os.makedirs(root)

sta = time.time()

for i in range(5):
    
    tissue_with_markers = novosparc.cm.Tissue(dataset=adata[test_indices[i], :],\
                                              locations=Y[train_indices[i]],\
                                              atlas_matrix=adata[train_indices[i], :].X)
    
    gene_names = tissue_with_markers.gene_names
    markers_in_sc = np.array([], dtype='int')
    insitu_genes = adata.var_names.tolist()
                        
    for marker in insitu_genes:
        marker_index = np.where(gene_names == marker)[0]
        if len(marker_index) > 0:
            markers_in_sc = np.append(markers_in_sc, marker_index[0])
                                                                        
    tissue_with_markers.setup_reconstruction(markers_to_use=markers_in_sc)
    tissue_with_markers.reconstruct(alpha_linear=0.5, epsilon=5e-3)
    
    gw = tissue_with_markers.gw
    pd.DataFrame(np.matmul(gw, loc.values[train_indices[i]]) * gw.shape[0]).to_csv('SI_Benchmarking/novosparc/pred_' + str(i) + '.csv', header = None)
                                                                                            
end = time.time()
print((end - sta) / 60.0, 'min')

Setting up for reconstruction ... done ( 2.72 seconds )
Reconstructing spatial information with 84 markers: 608 cells and 2431 locations ... 
Trying with epsilon: 5.00e-03


/home/linbuliao/anaconda3/lib/python3.9/site-packages/ot/bregman.py:517: UserWarning: Sinkhorn did not converge. You might want to increase the number of iterations `numItermax` or the regularization parameter `reg`.
  warnings.warn("Sinkhorn did not converge. You might want to "


Setting up for reconstruction ... done ( 2.84 seconds )
Reconstructing spatial information with 84 markers: 608 cells and 2431 locations ... 
Trying with epsilon: 5.00e-03


/home/linbuliao/anaconda3/lib/python3.9/site-packages/ot/bregman.py:517: UserWarning: Sinkhorn did not converge. You might want to increase the number of iterations `numItermax` or the regularization parameter `reg`.
  warnings.warn("Sinkhorn did not converge. You might want to "


Setting up for reconstruction ... done ( 2.92 seconds )
Reconstructing spatial information with 84 markers: 608 cells and 2431 locations ... 
Trying with epsilon: 5.00e-03


...

Setting up for reconstruction ... done ( 2.68 seconds )
Reconstructing spatial information with 84 markers: 608 cells and 2431 locations ... 
Trying with epsilon: 5.00e-03


/home/linbuliao/anaconda3/lib/python3.9/site-packages/ot/bregman.py:517: UserWarning: Sinkhorn did not converge. You might want to increase the number of iterations `numItermax` or the regularization parameter `reg`.
  warnings.warn("Sinkhorn did not converge. You might want to "


Setting up for reconstruction ... done ( 2.8 seconds )
Reconstructing spatial information with 84 markers: 607 cells and 2432 locations ... 
Trying with epsilon: 5.00e-03


/home/linbuliao/anaconda3/lib/python3.9/site-packages/ot/bregman.py:517: UserWarning: Sinkhorn did not converge. You might want to increase the number of iterations `numItermax` or the regularization parameter `reg`.
  warnings.warn("Sinkhorn did not converge. You might want to "


0.9097593148549398 min


# SpaOTsc

In [51]:
def corr2_coeff(A, B):
    # Rowwise mean of input arrays & subtract from input arrays themeselves
    A_mA = A - A.mean(1)[:, None]
    B_mB = B - B.mean(1)[:, None]

    # Sum of squares across rows
    ssA = (A_mA**2).sum(1)
    ssB = (B_mB**2).sum(1)

    # Finally get corr coeff
    return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))

In [55]:
root = 'SI_Benchmarking/SpaOTsc/'
if not os.path.exists(root):
    os.makedirs(root)

sta = time.time()

coords = Y
is_dmat_all = np.zeros((coords.shape[0], coords.shape[0]))
for j in range(coords.shape[0]):
    is_dmat_all[j] = (((coords[j] - coords)**2).sum(axis = 1) ** 0.5)

for i in range(5):
    
    print(i)
    
    df_sc = pd.DataFrame(adata[test_indices[i],].X.toarray(),
                         columns = adata[test_indices[i],].var_names.tolist(),
                         index = adata[test_indices[i],].obs_names.tolist())
    df_is = pd.DataFrame(adata[train_indices[i],].X.toarray(),
                         columns = adata[train_indices[i],].var_names.tolist(),
                         index = adata[train_indices[i],].obs_names.tolist())

    sc_pcc = np.corrcoef(df_sc)
    
    mcc = corr2_coeff(df_sc.values, df_is.values)
    C = np.exp(1-mcc)
    
    is_dmat = is_dmat_all[train_indices[i]][:, train_indices[i]]
        
    issc = SpaOTsc.spatial_sc(sc_data=df_sc, is_data=df_is,
                              sc_dmat = np.exp(1-sc_pcc),
                              is_dmat = is_dmat)
    
    mapping = issc.transport_plan(C**2, alpha=0.1, rho=100.0, epsilon=1.0, cor_matrix=mcc, scaling=False)
    
    pd.DataFrame(np.matmul(mapping, loc.values[train_indices[i]]) * mapping.shape[0]).to_csv('SI_Benchmarking/SpaOTsc/pred_' + str(i) + '.csv', header = None)
    
end = time.time()
print((end - sta) / 60.0, 'min')


0
1
2
3
4
9.09762298266093 min
